In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [2]:
class NeuralNetwork:
    def __init__(self, input_values, output, hidden_size, alfa = 0.02, epochs = 1000):
        self.X = input_values
        self.output = output
        self.hidden = hidden_size
        self.alfa = alfa
        self.erro = 0.001
        self.epochs = epochs
        self.m = self.X.shape[1]
        
        #Pesos hidden layer
        self.w1 = np.random.rand(self.X.shape[1], self.hidden) - 0.5
        self.bw1 = np.random.rand(1, self.hidden) - 0.5

        self.delta_inv = np.zeros((self.hidden, 1))
        self.dZ1 = np.zeros((self.hidden, 1))

        #Pesos camada de saída
        self.w2 = np.random.rand(self.hidden, self.output.shape[1]) - 0.5
        self.bw2 = np.random.rand(self.output.shape[1], 1) - 0.5
    
        self.dZ2 = np.zeros((self.output.shape[1], 1))

    def sigmoid_bipolar(self, z):
        return (2/(1+np.exp(-z))) - 1
    
    def sigmoid_partial(self, z2):
        return (0.5 *(1 + z2)*(1 - z2))
    
    def feed_foward(self, X):
       
        #Propagando entrada pela rede:
        
        self.z1 =  np.dot(X, self.w1) + self.bw1 #Xw + b
        self.a1 =  self.sigmoid_bipolar(self.z1)  #z = f(zin)
        
        #Cálculo saída da rede
        
        self.z2 = np.dot(self.a1, self.w2) + self.bw2 #y_in = ZW + Bw 
        self.yhat = self.sigmoid_bipolar(self.z2) #yhat = f(z_in)
        return self.yhat
    
    
    def retropropagation(self):
        
        #Não parar de calcular a regra da cadeia
        #Cálculo erro da camada de saída (Calculo deltinha_out)
        
        self.dyhat = (self.output - self.yhat) #dE/dYhat
        self.dZ2 = self.dyhat * self.sigmoid_partial(self.yhat) #dE/dZ2 = dE/dYhat * dYat/dZ2
        
        #Cálculo delta das hidden layers (Cálculo deltinha_in)
        
        self.da1 = self.dZ2.dot(self.w2.T) #dE/da1 = dE/dZ2 * dZ2/da1
        self.dZ1 = self.da1 * self.sigmoid_partial(self.a1) #dE/dZ1 = dE/da1 * da1/dZ1
        
        #Delta dos pesos do output layer
        
        self.dEdW2 = self.alfa * (self.a1.T.dot(self.dZ2)) #dE/dW2 = dE/dZ2 * dZ2/dW2 (a1)
        self.dEdBw2 = np.sum(self.alfa * self.dZ2, axis = 0)
    
        
        #Delta pesos da hiden layer
        
        self.dEdW1 = self.alfa * (self.X.T.dot(self.dZ1)) #dE/dW1 = dE/dZ1 * dZ1/dW1 (X)
        self.dEdBw1 = np.sum(self.alfa * self.dZ1, axis=0)
        
        #Pesos camada de saída
        
        self.w2 += self.dEdW2
        self.bw2 += self.dEdBw2
        
        #Pesos hidden layer
        
        self.w1 += self.dEdW1
        self.bw1 += self.dEdBw1
        
    def train(self, X_va, y_va):
        epoch = 0 
        self.lsm = []
        erro_total = 1
        self.acc_va = []
        self.acc_va.append(1)
        while(epoch < self.epochs) or (erro_total < self.erro):
            
            
            erro_total = 0
            yhat = self.feed_foward(self.X)
            self.retropropagation()
            
            erro_total += sum(0.5*((self.output - yhat)**2)) #Cálculo erro
            self.lsm.append(erro_total)
            
            #Avaliando o modelo com o validation set
            if epoch % 200 == 0:
                
                y_hat_va = self.predict(X_va)
                y_hat_va = self.output_encode(y_hat_va)
                cm_va = self.confusion_matrix(y_hat_va, y_va)
                self.acc_va.extend(self.accuracy(cm_va).tolist())
         
                
            epoch +=1
        
        tam = len(self.lsm)
        X_grid = np.array(np.arange(0, tam))
        plt.scatter(X_grid, self.lsm)
        plt.show()
        
    def predict(self, X):
       
        #Propagando entrada pela rede
            
        z1 =  np.dot(X, self.w1) + self.bw1 #z1 = XW1 + b
        a1 =  self.sigmoid_bipolar(z1)  #a1 = sigmoid(z1)
        
        #Cálculo saída da rede
        
        z2 = np.dot(a1, self.w2) + self.bw2 #z2 = ZW2 + Bw 
        yhat = self.sigmoid_bipolar(z2) #yhat = sigmoid(z2)
        return yhat
    
    def confusion_matrix(self, x1_in, x2_in):
        x1 = x1_in.copy()
        x2 = x2_in.copy()
        cm = np.zeros((2,2))
        
        if x1.shape != x2.shape:
            return print(f'{x1.shape} != {x2.shape}')
        else:
            for v1, v2 in zip(x1, x2):
                #print(f'{v1}, {v2}')
                if v1 == v2 and v1 == 1:
                    cm[[0], [0]] += 1
                    
                elif v1 == v2 and v1 == -1:
                    cm[[1], [1]] += 1
                    
                elif v1 != v2 and v1 == 1:
                    #print(f'FP = {v1}, {v2}')
                    cm[[0], [1]] += 1
                
                elif v1 != v2 and v1 == -1:
                    #print(f'FN = {v1}, {v2}')
                    cm[[1], [0]] += 1
            return cm

    def accuracy(self, cm_in):
        cm = cm_in.copy()
        return 100 * (cm[[0], [0]] + cm[[1], [1]])/np.concatenate(cm).sum()
    
    def output_encode(self, x_in):
        x = x_in.copy()
        for i in range(len(x)):
            x[i] = 1 if x[i] > 0 else -1
        return np.array(x).reshape((len(x), 1)).astype(int)
    
 

def categorical_encode(x, var):
    #x = x_in.copy()
    for i in range(len(x)):
        x[i] = 1 if x[i] == var else -1
    return np.array(x).reshape((len(x), 1)).astype(int)

def shuffle(a, b, random_seed):
    np.random.seed(random_seed) #Mantém o mesmo seed em random para manter os arrays indexados
    np.random.shuffle(a)
    np.random.seed(random_seed)
    np.random.shuffle(b)
    
def split_data(X, y, percentage, random_seed = 0):
    shuffle(X, y, random_seed)
    split_index = math.trunc((X.shape[0] * percentage)/100)
    X_train = X[:split_index,:]
    X_test = X[split_index:,:]
    y_train = y[:split_index,:]
    y_test = y[split_index:,:]
    return X_train, X_test, y_train, y_test

    



In [3]:
dataset = pd.read_csv('ds_sonar_vs_mines_UCI.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
y = categorical_encode(y, 'R')

X_train, X_test, y_train, y_test = split_data(X, y, 70, 42) 
X_test, X_validation, y_test, y_validation = split_data(X_test, y_test, 50, 42)

nn = NeuralNetwork(X_train, y_train, 30 , 0.01, 30000)
nn.train(X_validation, y_validation)
y_pred = nn.predict(X_test)
y_pred = nn.output_encode(y_pred)

cm = nn.confusion_matrix(y_pred, y_test)

accuracy = nn.accuracy(cm)


FileNotFoundError: [Errno 2] File ds_sonar_vs_mines_UCI.csv does not exist: 'ds_sonar_vs_mines_UCI.csv'

In [ ]:
accuracy

In [ ]:
cm

In [ ]:
nn.w1

In [ ]:
nn.w2